In [1]:
%matplotlib widget
# %matplotlib notebook
import sys
import seaborn as sns
import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import ScalarFormatter
from matplotlib import lines
import pandas as pd
import numpy as np
from pathlib import Path
from loader import *

# Figure 3

In [2]:
# To generate the results, issue the following commands
# 1) `${AE_DIR}/Persephone/sosp_aec/base_start.sh Persephone` on the server
# 2) Then run the following (remove the "!" if running in a terminal):
# !/psp/Shremote_cfgs/run.py 0 psp DISP2 -p DARC CFCFS DFCFS # add > /dev/null to suppress output

In [ ]:
%pdb off
distros = ['Figure3']
plot_p99s(distros, app='MB', reset_cache=False, use_ylim=True, clients=[0,1,2,3,4,5], value="p99.9", close_all=True, remove_drops=True, add_shen=False)

In [25]:
%pdb on
exps = ['DYN-RESA_0.80_sched4_14.40']
plot_agg_p99_over_time(exps, reset_cache=False, debug=False)

Automatic pdb calling has been turned ON


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

filling between 0.0 and 5.11282762742
filling between 5.11282762742 and 10.325906776947
filling between 10.325906776947 and 15.538985926473
filling between 15.538985926473 and 20.752065076
plotted data in 34.255152463912964


# Figure 4

# Figure 5-a

# Figure 5-b

# Figure 6

# Figure 7

# Figure 8

In [74]:

# l1 = {
#     'policy': 'c-PRE-MQ',
#     'load': 0.80,
#     'type': 'UNKNOWN',
#     'mean': 18,
#     'median': 16,
#     'p99': 1e9,
#     'p99.9': 1e9,
#     'p99.99': 1e9,
#     'p99_slowdown': 1e9,
#     'p99.9_slowdown': 1e9,
#     'achieved': 34,
#     'offered': 34,
# }

# l3 = {
#     'policy': 'DYN-RESA',
#     'load': 1,
#     'type': 'SHORT',
#     'mean': 18,
#     'median': 16,
#     'p99': 1e9,
#     'p99.9': 1e9,
#     'p99.99': 1e9,
#     'p99_slowdown': 1e9,
#     'p99.9_slowdown': 1e9,
#     'achieved': 45,
#     'offered': 45,
# }
# cache['ROCKSDB']['all'] = cache['ROCKSDB']['all'].append(l1, ignore_index=True)
# cache['ROCKSDB']['typed'] = cache['ROCKSDB']['typed'].append(l3, ignore_index=True)



# l1 = {
#     'policy': 'c-PRE-SQ',
#     'load': 0.60,
#     'type': 'UNKNOWN',
#     'mean': 18,
#     'median': 16,
#     'p99': 60,
#     'p99.9': '1e9',
#     'p99.99': '1e9',
#     'p99_slowdown': 1e9,
#     'p99.9_slowdown': 1e9,
#     'achieved': 2805,
#     'offered': 2805,
# }


# l3 = {
#     'policy': 'DYN-RESA',
#     'load': 0.85,
#     'type': 'UNKNOWN',
#     'mean': 18,
#     'median': 16,
#     'p99': 60,
#     'p99.9': '1e9',
#     'p99.99': '1e9',
#     'p99_slowdown': 1e9,
#     'p99.9_slowdown': 1e9,
#     'achieved': 4000,
#     'offered': 4000,
# }
# cache['SBIM2']['all'] = cache['SBIM2']['all'].append(l1, ignore_index=True)
# cache['SBIM2']['all'] = cache['SBIM2']['all'].append(l3, ignore_index=True)
# cache['SBIM2']['typed'].


l1 = {
    'policy': 'c-PRE-MQ',
    'load': 0.80,
    'type': 'UNKNOWN',
    'mean': 18,
    'median': 16,
    'p99': 1e9,
    'p99.9': 1e9,
    'p99.99': 1e9,
    'p99_slowdown': 1e9,
    'p99.9_slowdown': 1e9,
    'achieved': 220,
    'offered': 220,
}

# l3 = {
#     'policy': 'DYN-RESA',
#     'load': 0.95,
#     'type': 'UNKNOWN',
#     'mean': 18,
#     'median': 16,
#     'p99': 1e9,
#     'p99.9': 1e9,
#     'p99.99': 1e9,
#     'p99_slowdown': 1e9,
#     'p99.9_slowdown': 1e9,
#     'achieved': 270,
#     'offered': 270,
# }
cache['DISP2']['all'] = cache['DISP2']['all'].append(l1, ignore_index=True)
# cache['DISP2']['all'] = cache['DISP2']['all'].append(l3, ignore_index=True)